# Review Stockout Thresholds

This notebook has the objective of reviwing the results from filtering the monthly sales data based on the stockout thresholds chosen. 
To accomplish this, the notebook: 
1. Reports on the number of rows per department over time - only considering initialized items.
2. Reports on the number of rows after filtering for stockouts.
3. Provides with summary statisticks for the stockout vs non-stockout distributions.
4. Provides with plots for the stockout vs non-stockout distributions.

**Note:** To simplify the analysis I will define "over time" as "per year".


Note to self: Before I tackle this I need to know that the filter for "initialized items" is working as expected. I will put this on hold as I prioritize said filter.


In [8]:
# --- Imports ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy.stats import ks_2samp
import ruptures as rpt
import sys
import os

# --- Imports from project root ---
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..", "..",))
sys.path.append(project_root)
from src.core.clients.bigquery import BigQueryClient

# --- Plotting style ---
sns.set(style='whitegrid', context='notebook')

ModuleNotFoundError: No module named 'src'

In [9]:
# --- Load Data From BigQuery ---
from src.core.clients.bigquery import BigQueryClient

bq_client = BigQueryClient()

sales_query = """
    SELECT 
        tgt.ctx_date_month,
        tgt.ctx_cat_id, 
        tgt.ctx_dept_id, 
        tgt.ctx_item_id,
        tgt.tgt_monthly_sales,
        tgt.tgt_monthly_sales_sum_3_next_months,
        stckt.is_stockout_tgt
    FROM `porygon-pipelines.walmart_targets_and_filters.target_store_item_monthly` tgt
    LEFT JOIN `porygon-pipelines.walmart_targets_and_filters.stockout_filter` stckt
        ON tgt.ctx_item_id = stckt.ctx_item_id
        AND tgt.ctx_store_id = stckt.ctx_store_id
        AND tgt.ctx_date_month = stckt.ctx_date_month
    WHERE 
        tgt.ctx_store_id = 'CA_1'
"""


# Extract data
df = bq_client.load_from_query(sales_query)

ModuleNotFoundError: No module named 'src'

In [10]:
# --- Data Preparation ---
# Define target name to analyze:
target_name = "tgt_monthly_sales_sum_3_next_months"

# Create a df that only includes initialized items
df_init = df[~df['is_stockout_tgt'].isna()]

# Print summary dataframe shapes:
print(f"Full dataframe shape: {df.shape}")
print(f"Initialized items dataframe shape: {df_init.shape}")

# Create column to denote year.
df_init["year"] = pd.DatetimeIndex(df_init['ctx_date_month']).year

NameError: name 'df' is not defined

In [11]:
df[df['is_stockout_tgt'].isna()]

NameError: name 'df' is not defined

# 0.1 Report on the numberof prows per item department

This isonly considering initialized items. These summary statistics I want to see them using both summary statistics and plots.

In [13]:
# Group by year and department, count months per group
grouped = (
    df_init
    .groupby(["year", "ctx_dept_id"])
    .agg(month_count=("ctx_date_month", "count"))
    .reset_index()
)

# Pivot to get years as rows and departments as columns
dept_row_count_per_year = grouped.pivot(
    index="year",
    columns="ctx_dept_id",
    values="month_count"
)

# Note that Since
dept_row_count_per_year

ctx_dept_id,FOODS_1,FOODS_2,FOODS_3,HOBBIES_1,HOBBIES_2,HOUSEHOLD_1,HOUSEHOLD_2
year,,,,,,,
2011,2592,4776,9876,4992,1788,6384,6180
2012,2592,4776,9876,4992,1788,6384,6180
2013,2592,4776,9876,4992,1788,6384,6180
2014,2592,4776,9876,4992,1788,6384,6180
2015,2592,4776,9876,4992,1788,6384,6180
2016,216,398,823,416,149,532,515
